In [ ]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Streaming from Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.1")
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

In [4]:
# Create the kafka_df to read from kafka as a stream

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "cf-kafka-1:29092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

# Create the kafka_df to read from kafka in batch mode

kafka_df = (
    spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers", "cf-kafka-1:29092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

In [5]:
# View schema for raw kafka_df
kafka_df.printSchema()
# kafka_df.show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|NULL|[7B 22 65 76 65 6...|device-data|        0|     0|2025-09-12 15:12:...|            0|
|NULL|[7B 22 65 76 65 6...|device-data|        0|     1|2025-09-12 15:16:...|            0|
|NULL|       [74 65 73 74]|device-data|        0|     2|2025-09-12 15:37:...|            0|
|NULL|[7B 22 65 76 65 6...|device-data|        0|     3|2025-09-12 15:41:...|            0|
|NULL|[7B 22 65 76 65 6...|device-data|        0|     4|2025-09-12 15:56:

In [9]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))
# kafka_json_df.show()

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|NULL|{"eventId": "ba2e...|device-data|        0|     0|2025-09-12 15:12:...|            0|
|NULL|{"eventId": "e3cb...|device-data|        0|     1|2025-09-12 15:16:...|            0|
|NULL|                test|device-data|        0|     2|2025-09-12 15:37:...|            0|
|NULL|{"eventId": "e3cb...|device-data|        0|     3|2025-09-12 15:41:...|            0|
|NULL|{"eventId": "e3cb...|device-data|        0|     4|2025-09-12 15:56:...|            0|
|NULL|{"eventId": "e3cb...|device-data|        0|     5|2025-09-12 16:17:...|            0|
|NULL|{"eventId": "e3cb...|device-data|        0|     6|2025-09-12 16:18:...|            0|
|NULL|{"eventId": "aa90...|device-data|        0|     7|2025-09-12 16:18:...|   

In [7]:
# Schema of the Payload

from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = (
    StructType(
    [StructField('customerId', StringType(), True), 
    StructField('data', StructType(
        [StructField('devices', 
                     ArrayType(StructType([ 
                        StructField('deviceId', StringType(), True), 
                        StructField('measure', StringType(), True), 
                        StructField('status', StringType(), True), 
                        StructField('temperature', LongType(), True)
                    ]), True), True)
        ]), True), 
    StructField('eventId', StringType(), True), 
    StructField('eventOffset', LongType(), True), 
    StructField('eventPublisher', StringType(), True), 
    StructField('eventTime', StringType(), True)
    ])
)

In [8]:
# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json,col

# Flatten the value column into values_json column
streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema))
streaming_df.printSchema()
# streaming_df.show(truncate=False)

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- values_json: struct (nullable = true)
 |    |-- customerId: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- devices: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |    |-- measure: string (nullable = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- temperature: long (nullable = true)
 |    |-- eventId: string (nullable = true)
 |    |-- eventOffset: long (nullable = true)
 |    |-- eventPublisher: string (nullable = true)
 |    |-- eventTime: string (nullable = true)

+----+-------------------------------------------------

In [12]:
# Drops all columns except those in values_json using the selectExpr
streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
# streaming_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)

+----------+------------------------------------------------+------------------------------------+-----------+--------------+--------------------------+
|customerId|data                                            |eventId                             |eventOffset|eventPublisher|eventTime                 |
+----------+------------------------------------------------+------------------------------------+-----------+--------------+----

In [13]:
# Explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode

exploded_df = streaming_df.withColumn("data_devices", explode("data.devices"))
# exploded_df.show()

+----------+--------------------+--------------------+-----------+--------------+--------------------+--------------------+
|customerId|                data|             eventId|eventOffset|eventPublisher|           eventTime|        data_devices|
+----------+--------------------+--------------------+-----------+--------------+--------------------+--------------------+
|   CI00103|{[{D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|{D001, C, ERROR, 15}|
|   CI00103|{[{D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|{D002, C, SUCCESS...|
|   CI00103|{[{D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|{D001, C, ERROR, 15}|
|   CI00103|{[{D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|{D002, C, SUCCESS...|
|   CI00103|{[{D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|{D001, C, ERROR, 15}|
|   CI00

In [29]:
# Check the schema of the exploded_df, place a sample json file and change readStream to read 
exploded_df.printSchema()
#exploded_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- data_devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [30]:
# Flatten the exploded df
from pyspark.sql.functions import col

flattened_df = (
    exploded_df
    .drop("data")
    .withColumn("deviceId", col("data_devices.deviceId"))
    .withColumn("measure", col("data_devices.measure"))
    .withColumn("status", col("data_devices.status"))
    .withColumn("temperature", col("data_devices.temperature"))
    .drop("data_devices")
)


In [31]:
# Check the schema of the flattened_df, place a sample json file and change readStream to read 
flattened_df.printSchema()
#flattened_df.show(truncate=False)

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)



In [ ]:
# Write the output to console sink to check the output

(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
 .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())

In [ ]:
# Check the data at the output location

